In [1]:
import urllib.request
urllib.request.urlretrieve('http://www.atarimania.com/roms/Roms.rar','Roms.rar')
!pip install unrar
!unrar x Roms.rar
!mkdir rars
!mv HC\ ROMS.zip   rars
!mv ROMS.zip  rars
!python -m atari_py.import_roms rars


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from Roms.rar

Extracting  HC ROMS.zip                                                   36%  OK 
Extracting  ROMS.zip                                                      74% 99%  OK 
All OK
copying adventure.bin from ROMS/Adventure (1980) (Atari, Warren Robinett) (CX2613, CX2613P) (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/adventure.bin
copying air_raid.bin from ROMS/Air Raid (Men-A-Vision) (PAL) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/air_raid.bin
copying alien.bin from ROMS/Alien (1982) (20th Century Fox Video Games, Douglas 'Dallas North' Neubauer) (11006) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/alien.bin
copying amidar.bin from ROMS/Amidar (1982) (Parker Brothers, Ed Temple) (PB5310) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/amidar.bin
copying assault.bin from ROMS/Assau

In [3]:
!pip install --user keras-rl
!pip install --user keras==2.2.4
!pip install --user Pillow
!pip install --user h5py

     |████████████████████████████████| 40kB 7.4MB/s 
  Created wheel for keras-rl: filename=keras_rl-0.4.2-cp37-none-any.whl size=48382 sha256=c91551112dc93fc8e4337db8e9a0f111e49d49dd01e2ad8884c29e1d18125b26
  Stored in directory: /root/.cache/pip/wheels/7d/4d/84/9254c9f2e8f51865cb0dac8e79da85330c735551d31f73c894
Successfully built keras-rl
     |████████████████████████████████| 317kB 35.6MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 


In [5]:
!pip install  gym keras-rl2 gym[atari]

     |████████████████████████████████| 61kB 9.5MB/s 


In [6]:
import numpy as np
import gym
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor


In [7]:
# Get the environment and extract the number of actions and the input shape.
env = gym.make('Breakout-v0')
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n
print(nb_actions)
input_shape_ = env.observation_space.shape
env.unwrapped.get_action_meanings()

4


['NOOP', 'FIRE', 'RIGHT', 'LEFT']

In [8]:

""" contains a model to train dqn agent for atari breakout game"""
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4


class AtariProcessor(Processor):
    """ processor for the game"""

    def process_observation(self, observation):
        """process images taken """
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert(
            'L')  # resize and convert to grayscale
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        # saves storage in experience memory
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        """ normalization of the batch images"""
        # We could perform this processing step in `process_observation`.
        # In this case, however, we would need to store a `float32` array
        # instead, which is 4x more memory intensive than
        # an `uint8` array. This matters if we store 1M observations.
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        """limit values in reward array between -1 and 1 """
        return np.clip(reward, -1., 1.)


In [11]:


def build_model(input_shape, nb_actions):
  # input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
  model = Sequential()
  # if K.image_data_format() == 'channels_last':
  #     # (width, height, channels)
  #     model.add(Permute((2, 3, 1), input_shape=input_shape))
  # elif K.image_data_format() == 'channels_first':
  #     # (channels, width, height)
  #     model.add(Permute((1, 2, 3), input_shape=input_shape))

  model.add(Convolution2D(32, (8, 8), strides=(4, 4), input_shape=input_shape))
  model.add(Activation('relu'))
  model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
  model.add(Activation('relu'))
  model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
  model.add(Activation('relu'))
  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dense(nb_actions))
  model.add(Activation('linear'))
  return model

model = build_model(input_shape=((WINDOW_LENGTH,) + input_shape_), nb_actions=nb_actions)

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 4, 51, 39, 32)     6176      
_________________________________________________________________
activation_5 (Activation)    (None, 4, 51, 39, 32)     0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 24, 18, 64)     32832     
_________________________________________________________________
activation_6 (Activation)    (None, 4, 24, 18, 64)     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 4, 22, 16, 64)     36928     
_________________________________________________________________
activation_7 (Activation)    (None, 4, 22, 16, 64)     0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 90112)            

In [12]:

def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=100000, window_length=4)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [ ]:
def train(model, nb_actions):
  dqn = build_agent(model, nb_actions)

  dqn.compile(Adam(lr=1e-3), metrics=['mae'])
  filename = 'policy.h5'
  dqn.fit(env,
          nb_steps=10000,
          log_interval=1000, visualize=False, verbose=2)
  # After training is done, we save the final weights one more time.
  dqn.save_weights(filename, overwrite=True)

train(model=model, nb_actions=nb_actions)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Training for 100000 steps ...


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


   274/100000: episode: 1, duration: 3.746s, episode steps: 274, steps per second:  73, episode reward:  2.000, mean reward:  0.007 [ 0.000,  1.000], mean action: 1.438 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   605/100000: episode: 2, duration: 4.480s, episode steps: 331, steps per second:  74, episode reward:  2.000, mean reward:  0.006 [ 0.000,  1.000], mean action: 1.517 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   779/100000: episode: 3, duration: 2.439s, episode steps: 174, steps per second:  71, episode reward:  0.000, mean reward:  0.000 [ 0.000,  0.000], mean action: 1.506 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   957/100000: episode: 4, duration: 2.476s, episode steps: 178, steps per second:  72, episode reward:  0.000, mean reward:  0.000 [ 0.000,  0.000], mean action: 1.500 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


  1187/100000: episode: 5, duration: 142.785s, episode steps: 230, steps per second:   2, episode reward:  1.000, mean reward:  0.004 [ 0.000,  1.000], mean action: 1.491 [0.000, 3.000],  loss: 911.491444, mae: 69.404943, mean_q: 98.318977, mean_eps: 0.901585
  1502/100000: episode: 6, duration: 238.566s, episode steps: 315, steps per second:   1, episode reward:  2.000, mean reward:  0.006 [ 0.000,  1.000], mean action: 1.524 [0.000, 3.000],  loss: 14.762260, mae: 67.658430, mean_q: 91.554720, mean_eps: 0.879040
  1739/100000: episode: 7, duration: 177.731s, episode steps: 237, steps per second:   1, episode reward:  1.000, mean reward:  0.004 [ 0.000,  1.000], mean action: 1.633 [0.000, 3.000],  loss: 19.846776, mae: 67.330377, mean_q: 91.617576, mean_eps: 0.854200
  1916/100000: episode: 8, duration: 133.405s, episode steps: 177, steps per second:   1, episode reward:  0.000, mean reward:  0.000 [ 0.000,  0.000], mean action: 1.469 [0.000, 3.000],  loss: 13.380892, mae: 67.343022, m

In [ ]:
!pip install colabgymrender
from colabgymrender.recorder import Recorder

def play(model, nb_actions):
  dqn = build_agent(model, nb_actions)
  dqn.load_weights(weights_filename)
  dqn.test(env, nb_episodes=10, visualize=True)

  """recorde and visualization the game"""
  directory = './video'
  env = Recorder(env, directory)

  observation = env.reset()
  terminal = False
  while not terminal:
    action = env.action_space.sample()
    observation, reward, terminal, info = env.step(action)
  env.play()

play(model=model, nb_actions=nb_actions)